# Basic Statistics from Movie DF

**Metadata**

| **#** | **Column**               | **Description**                                                                                    |
|-------|--------------------------|----------------------------------------------------------------------------------------------------|
| 1     | **title**                | The official title of the movie.                                                                   |
| 2     | **release_year**         | The year when the movie was officially released.                                                   |
| 3     | **genres**               | Categories of genres the movie belongs to (can have multiple values, separated by commas).         |
| 4     | **director**             | Director(s) of the movie.                                                                          |
| 5     | **runtime**              | Duration of the movie in minutes.                                                                  |
| 6     | **language**             | The language in which the movie was originally produced.                                           |
| 7     | **original_title**       | The title of the movie in its original language.                                                   |
| 8     | **popularity**           | TMDB's score based on votes, views, searches, social engagement, and others.                       |
| 9     | **events**               | Describes events or triggers within the movie (e.g., violence, strong language).                   |
| 10    | **imdb_rating**          | Average rating of the movie on IMDb, on a scale from 0 to 10.                                      |
| 11    | **imdb_votes**           | Number of votes that contributed to the IMDb rating.                                               |
| 12    | **tmdb_rating**          | Rating of the movie on TMDB, on a scale from 0 to 10.                                              |
| 13    | **tmdb_votes**           | Number of votes that contributed to the TMDB rating.                                               |
| 14    | **budget**               | Financial budget allocated for the movie production.                                               |
| 15    | **revenue**              | Box office earnings of the movie.                                                                  |
| 16    | **profit**               | Profit of the movie, calculated as `revenue - budget`.                                             |


In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append('../utils')
import functions

In [3]:
movies = pd.read_csv('../data/clean/movie_df.csv')
display(movies)

,title,release_year,genres,director,runtime,language,original_title,popularity,events,imdb_rating,imdb_votes,tmdb_rating,tmdb_votes,budget,revenue,profit
0,Bad Boys for Life,2020,"thriller, action, crime","Bilall Fallah, Adil El Arbi",124,English,Bad Boys for Life,67.7,"flashing lights or images, shaving or cutting,...",6.5,186716,7.1,8251,90000000,426505244,336505244
1,Avatar: The Way of Water,2022,"science fiction, adventure, action",James Cameron,192,English,Avatar: The Way of Water,158.5,"kids dying, jump scares, people being burned a...",7.5,518001,7.6,11946,460000000,2320250281,1860250281
2,Pet Sematary,2019,"horror, thriller","Dennis Widmyer, Kevin Kölsch",100,English,Pet Sematary,26.1,"dogs dying, kids dying, jump scares, people be...",5.7,100618,5.7,3186,21000000,113118226,92118226
3,How to Train Your Dragon: The Hidden World,2019,"animation, family, adventure",Dean DeBlois,104,English,How to Train Your Dragon: The Hidden World,207.5,"needles or syringes are used, dragons dying, f...",7.4,158269,7.8,6381,129000000,524580592,395580592
4,Star Wars: The Rise of Skywalker,2019,"adventure, action, science fiction",J.J. Abrams,142,English,Star Wars: The Rise of Skywalker,63.2,"jump scares, people being burned alive, flashi...",6.4,507321,6.3,9886,416000000,1074144248,658144248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Longlegs,2024,"horror, thriller, crime",Osgood Perkins,101,English,Longlegs,153.9,"kids dying, jump scares, flashing lights or im...",6.7,141019,6.6,1475,10000000,126388179,116388179
709,Piece by Piece,2024,"animation, music, documentary",Morgan Neville,94,English,Piece by Piece,34.0,"parents dying, black guy dies first, babies/un...",6.9,3210,7.8,59,16000000,10291872,-5708128
710,Moana 2,2024,"animation, adventure, family, comedy","David G. Derrick Jr., Dana Ledoux Miller, Jaso...",100,English,Moana 2,4485.0,"flashing lights or images, ghosts, bugs, restr...",7.1,30852,6.8,424,150000000,600055655,450055655
711,Sound of Hope: The Story of Possum Trot,2024,drama,Joshua Weigel,130,English,Sound of Hope: The Story of Possum Trot,58.5,"hate speech, child abuse, minority misrepresen...",7.1,1782,6.7,23,8500000,11721425,3221425
